# Tutorial 14: Use Copilot to generate dialogues with SDialog


We will ask to Copilot or Cursor to create the code for us:

```plain
#fetch https://sdialog.readthedocs.io/en/latest/llm.txt

Write a Python code to generate dialog between an patient and a polite doctor using sdialog with multi-agent support, using Amazon bedrock call to Claude 3.5 sonnet
```

The output:

In [ ]:
!pip install boto3 langchain-aws

In [ ]:
import os
import sdialog
from sdialog import Context 
from sdialog.agents import Agent
from sdialog.personas import Persona

AWS configuration with your usual method:

In [ ]:
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "XXX"

Configure sdialog to use your model:

In [ ]:
sdialog.config.llm("amazon:anthropic.claude-3-5-sonnet-20240620-v1:0", 
                   temperature=0.7, 
                   max_tokens=512,
                   region_name="us-east-1")

Define the characters:

In [ ]:
patient = Persona(
    name="Marie Dubois",
    age=45,
    gender="female",
    role="patient",
    background="Marie is a 45-year-old patient who is consulting for recurring headaches and fatigue. She works in an office and has two teenage children.",
    personality="Marie is polite, a bit anxious, and tends to ask many questions. She worries easily about her health.",
    circumstances="She made an appointment because her symptoms have persisted for two weeks and are affecting her work.",
    rules="She must be respectful towards the doctor and express her concerns clearly."
)

doctor = Persona(
    name="Dr. Pierre Martin",
    age=52,
    gender="male", 
    role="general practitioner",
    background="Dr. Martin is an experienced general practitioner with 25 years of experience. He is known for his empathetic approach and patience with his patients.",
    personality="He is very professional, empathetic, and takes time to listen to his patients. He explains things clearly and reassuringly.",
    circumstances="He practices in his private office and has a holistic approach to medicine.",
    rules="He must be polite, professional, ask relevant questions, and reassure the patient while remaining medically precise."
)

Create the consultation context:

In [ ]:
context = Context(
    location="Dr. Martin's medical office",
    datetime="2024-01-15 14:30",
    environment="A modern and welcoming medical office with a calm and professional atmosphere",
    topics=["medical consultation", "headaches", "fatigue", "diagnosis"],
    goals=["establish a diagnosis", "reassure the patient", "propose treatment"],
    constraints=["respect medical confidentiality", "be professional", "limit consultation to 20 minutes"]
)

Create the agents with your model:

In [ ]:
agent_patient = Agent(
    persona=patient,
    name="Marie",
    context=context,
    first_utterance="Hello doctor, thank you for seeing me. I came to see you because I've had persistent headaches for two weeks, and I feel very tired.",
    dialogue_details="Medical consultation for persistent symptoms",
    response_details="Natural and respectful responses, expressing patient concerns"
)

agent_doctor = Agent(
    persona=doctor,
    name="Dr. Martin", 
    context=context,
    dialogue_details="Professional medical consultation",
    response_details="Professional, empathetic and reassuring medical responses"
)

Generate the dialogue:

In [ ]:
dialogue = agent_patient.dialog_with(
    agent_doctor,
    context=context,
    max_turns=12,  # 6 exchanges (12 turns total)
    seed=42  # For reproducibility
)

dialogue.print()

print(f"\n=== Dialog Statistic ===")
print(f"Number of turns: {len(dialogue)}")
print(f"Number of words: {dialogue.length('words')}")
print(f"Estimated duration: {dialogue.length('minutes')} minutes")
print(f"Model used: {dialogue.model}")
print(f"Seed: {dialogue.seed}")

Save the dialogue:

In [ ]:
dialogue.to_file("demo_dialog_doctor_patient.json")

# Second use case: Customer service

You can do the same with customer service:
```plain
#fetch https://sdialog.readthedocs.io/en/latest/llm.txt

Write a Python code to generate dialog between an angry customer and a polite support agent using sdialog with multi-agent support, using Amazon bedrock call to Claude 3.5 sonnet
```

In [ ]:
sdialog.config.llm("amazon:anthropic.claude-3-5-sonnet-20240620-v1:0", 
                   temperature=0.7, 
                   max_tokens=512,
                   region_name="us-east-1")

In [ ]:
# Angry customer persona
angry_customer = Persona(
    name="John Smith",
    age=35,
    gender="male",
    role="frustrated customer",
    background="John recently purchased a defective product a week ago. He has already contacted support multiple times without getting a satisfactory solution.",
    personality="John is frustrated, impatient, and angry. He tends to raise his voice and be direct in his complaints. He feels ignored by the company.",
    circumstances="He is calling support for the third time this week, determined to get an immediate refund or replacement.",
    rules="He must express his frustration clearly but avoid using offensive language. He should insist that he has already contacted support multiple times."
)

# Polite support agent persona
support_agent = Persona(
    name="Sarah Johnson",
    age=28,
    gender="female", 
    role="customer support agent",
    background="Sarah is an experienced customer support agent with 5 years of experience. She is known for her patience and ability to solve customer problems.",
    personality="She remains calm and professional even when dealing with difficult customers. She is empathetic, patient, and solution-oriented.",
    circumstances="She is handling John's call and aims to resolve his issue satisfactorily while maintaining a professional attitude.",
    rules="She must remain polite and professional, apologize for inconveniences, and offer appropriate solutions. She should never lose her composure."
)

In [ ]:
context = Context(
    location="Customer Support Call Center",
    datetime="2024-01-15 14:30",
    environment="A quiet office with minimal background noise. The agent works from her support station.",
    topics=["product issue", "customer dissatisfaction", "refund request", "customer support"],
    goals=["resolve the customer's complaint", "offer a solution", "ensure customer satisfaction"],
    constraints=["follow company policies", "maintain professionalism", "limit call to 15 minutes"]
)

In [ ]:
customer_agent = Agent(
    persona=angry_customer,
    name="John",
    context=context,
    first_utterance="Hello, this is John Smith. I'm calling for the third time this week about my defective product and no one is helping me! This is unacceptable!",
    dialogue_details="Customer expressing frustration about a defective product",
    response_details="Angry and impatient responses demanding immediate action"
)

support_agent_polite = Agent(
    persona=support_agent,
    name="Sarah",
    context=context,
    dialogue_details="Support agent handling a customer complaint",
    response_details="Calm and polite responses aiming to resolve the issue"
)

In [ ]:
dialogue = customer_agent.dialog_with(
    support_agent_polite,
    context=context,
    max_turns=14,  # 7 exchanges (14 turns total)
    seed=42  # For reproducibility
)

dialogue.print()

print(f"\n=== Dialog Statistic ===")
print(f"Number of turns: {len(dialogue)}")
print(f"Number of words: {dialogue.length('words')}")
print(f"Estimated duration: {dialogue.length('minutes')} minutes")
print(f"Model used: {dialogue.model}")
print(f"Seed: {dialogue.seed}")

In [ ]:
dialogue.to_file("customer_support_dialogue.json")

## Generate audio dialogs

In [ ]:
audio_dialogue = dialogue.to_audio()

In [ ]:
display(Audio(audio_dialogue.audio_step_1_filepath, autoplay=True))